# EDA

In [44]:
from pprint import pprint
import pandas as pd
import numpy as np
import re

## LOAD DATA

In [43]:
df = pd.read_csv('data/train.tsv',sep='\t')
# keep = set(['item_id','Item Class ID','Product Long Description','Product Name',
#             'Product Short Description','Short Description','tag'])
keep = set(['Product Long Description','Product Name','tag'])
drop = set(df.columns) - keep
df.drop(drop,axis=1,inplace=True)
df.dropna(inplace=True)
df.head(10)

,Product Long Description,Product Name,tag
0,<b>Interfaces/Ports</b><li>Network (RJ-45): ...,Cyberpower Metered ATS PDU 200-240V 20A 1U 10-...,[4483]
1,General InformationThe SmartMount&reg; Univers...,Peerless SmartMount SF650P Universal Flat Wall...,[581514]
2,"Control Transformer, VA Rating 50VA, Input Vol...","DAYTON 31EH66 Control Transformer Fused,4.02H ...",[4483]
3,<b>General Information</b><li>Product Type: ...,STEREN-CUSTOM INSTALL 255-047 STEREN MINI TO R...,[4537]
4,"<br><b>LG 47LN5700 47"" 1080p 120Hz LED HDTV:</...","LG 47LN5700 47"" 1080p 120Hz LED (3.17"" ultra-s...","[1229817, 1229821]"
5,Features- Transformer Ground Connectors are us...,Morris 91902 Transformer Ground Connector,[4537]
6,Take a look at billionaire adventurer Reese Ba...,Newsreaders: Billionaire Goes to Hell / Sitcom...,[4483]
7,Power Grip RV Power Cord Adapter - These elect...,Camco Mfg. 15m-30f Electric Adapter 55223,[4483]
8,TV should be two things: smart and beautiful. ...,SAMSUNG UN48H6400 48in 1080p 120Hz Apps QuadCo...,[3304195]
9,<P><FONT size=2 face=Verdana>Prepare yourself ...,LG 55UF7600 - 55-inch 2160p 120Hz 4K UHD LED T...,[1180168]


## ORIGINAL TAG COUNTS/FREQUENCY

In [66]:
df.tag.value_counts()

[4537]                                                   2667
[4483]                                                   2002
[581514]                                                  730
[529295]                                                  513
[95987]                                                   401
[106546]                                                  388
[447913]                                                  379
[4538]                                                    244
[127175]                                                  241
[4536]                                                    220
[1229821]                                                 201
[650659]                                                  189
[1180168]                                                 175
[522484]                                                  169
[1229817]                                                 154
[1070524]                                                 146
[3304195

## INDIVIDUAL ONLY TAG COUNTS/FREQUENCY

In [25]:
df.tag.value_counts()

[4537]                                  2667
[4483]                                  2002
[581514]                                 730
[529295]                                 513
[95987]                                  401
[106546]                                 388
[447913]                                 379
[4538]                                   244
[127175]                                 241
[4536]                                   220
[1229821]                                201
[650659]                                 189
[1180168]                                175
[522484]                                 169
[1229817]                                154
[1070524]                                146
[5065]                                    80
[1071165]                                 49
[1085065]                                 47
[1225174]                                 47
[1229820]                                 23
[62056]                                   22
[3304195] 

## Find the 32 unique shelve tags

In [182]:
tags = []
for tag in df.tag:
    t = tag.replace('[','')
    t = t.replace(']','')
    t = t.replace(' ','',10)
    t = t.split(',')
    if len(t) == 1:
        tags.extend(t)
tags.extend(['1229823']) #tag that is never by itself...
unique_tags = ['['+tag+']' for tag in set(tags)]
print(unique_tags)
print ''
print 'total shelves found ==>', len(unique_tags)

['[1071165]', '[127175]', '[529295]', '[106546]', '[3304195]', '[4538]', '[645319]', '[4536]', '[4537]', '[1084835]', '[1229818]', '[1229819]', '[1229817]', '[5065]', '[95987]', '[1085065]', '[1229823]', '[1070524]', '[1229820]', '[447913]', '[650659]', '[522484]', '[648819]', '[1225174]', '[62056]', '[1180168]', '[1229825]', '[133270]', '[1229821]', '[4483]', '[4457]', '[581514]']

total shelves found ==> 32


## Unique count per tag 

In [183]:
unique_tag_count = {key:0 for key in unique_tags}
count = 0
for tag in df.tag:
    if tag in unique_tag_count.keys():
        unique_tag_count[tag] += 1
        count += 1
print unique_tag_count
print count

{'[4483]': 2002, '[1229818]': 4, '[1225174]': 47, '[650659]': 189, '[1071165]': 49, '[4537]': 2667, '[5065]': 80, '[1229823]': 0, '[62056]': 22, '[1085065]': 47, '[529295]': 513, '[1229825]': 13, '[1229821]': 201, '[4457]': 19, '[648819]': 6, '[3304195]': 21, '[581514]': 730, '[106546]': 388, '[1229819]': 18, '[1070524]': 146, '[127175]': 241, '[4536]': 220, '[447913]': 379, '[1229817]': 154, '[645319]': 3, '[95987]': 401, '[1180168]': 175, '[133270]': 3, '[1084835]': 21, '[522484]': 169, '[4538]': 244, '[1229820]': 23}
9195


## Total count/mentions of all 32 shelve tags

In [102]:
keys = ['1071165', '127175', '529295', '106546', '3304195', '4538', '645319', '4536',
        '4537', '1084835', '1229818', '1229819', '1229817', '5065', '95987', '1085065', 
        '1229823', '1070524', '1229820', '447913', '650659', '522484', '648819', '1225174', 
        '62056', '1180168', '1229825', '133270', '1229821', '4483', '4457', '581514']
vals = [0]*32
d_total = dict(zip(keys,vals))
total_count = 0
for tag in df.tag:
    for key in keys:
        if re.search(key,tag):
            d_total[key] += 1
            total_count += 1
print d_total
print ''
print 'total tag mentions (including duplicates) ==> ', total_count
print 'num samples ==> ', df.shape[0]

{'1071165': 52, '127175': 510, '529295': 520, '106546': 825, '3304195': 345, '4538': 250, '645319': 3, '4536': 229, '4537': 2801, '1084835': 21, '1229818': 16, '1229819': 60, '1229817': 389, '5065': 326, '95987': 619, '1085065': 51, '1070524': 169, '4483': 2110, '447913': 608, '650659': 195, '522484': 603, '648819': 11, '1225174': 53, '62056': 29, '1180168': 271, '1229825': 50, '1229823': 3, '133270': 3, '1229821': 763, '1229820': 175, '4457': 22, '581514': 771}

total tag mentions (including duplicates) ==>  12853
num samples ==>  10584


## Create dictionary w/ respect to each unique shelf tag
* total count (all mentions)
* frequency of other tags
* frequency of tag combos

In [178]:
d_tag_info = defaultdict(dict)
for key in keys:
    temp_list1 = []
    temp_list2 = []
    temp_dict1 = {}
    temp_dict2 = {}
    temp_dict3 = {}
    
    # list of all tag/tag combos
    for tag in df.tag:
        if re.search(key,tag):
            temp_list1.append(tag)

    # freq of all tag combos        
    for tag in temp_list1:
        if tag in temp_dict1.keys():
            temp_dict1[tag] += 1
        else:
            temp_dict1[tag] = 1
            
    # unique tags        
    for tag in temp_dict1.keys():
        t = tag.replace('[','')
        t = t.replace(']','')
        t = t.replace(' ','',10)
        t = t.split(',')
        temp_list2.extend(t)
    unique_tags = list(set(temp_list2))
    
    # freq of unique tags
    for unique_tag in unique_tags:
        for all_tag in temp_list1:
            try:
                if re.search(unique_tag,all_tag):
                    temp_dict2[unique_tag] += 1
            except:
                temp_dict2[unique_tag] = 1

    # tag info            
    temp_dict3['total'] = d_total[key]
    temp_dict3['tag_freq'] = temp_dict2
    temp_dict3['combo_freq'] = temp_dict1
    d_tag_info[key] = temp_dict3

## Review tag freqency and combinations

In [179]:
for key,val in d_tag_info.items():
    print ''
    print ''
    print 'tag: {}'.format(key)
    print 'total: {}'.format(val['total'])
    print 'tag freq ==> {}'.format(sorted(val['tag_freq'].items(),key=lambda x: x[1],reverse=True))
    print 'combo freq ==> {}'.format(sorted(val['combo_freq'].items(),key=lambda x: x[1],reverse=True))



tag: 1071165
total: 52
tag freq ==> [('1071165', 52), ('447913', 2), ('106546', 2), ('127175', 1)]
combo freq ==> [('[1071165]', 49), ('[1071165, 106546]', 1), ('[447913, 127175, 106546, 1071165]', 1), ('[1071165, 447913]', 1)]


tag: 127175
total: 510
tag freq ==> [('127175', 510), ('522484', 185), ('106546', 184), ('95987', 41), ('4483', 14), ('529295', 4), ('4537', 3), ('447913', 2), ('3304195', 2), ('1071165', 1), ('648819', 1), ('1225174', 1), ('1070524', 1), ('1229825', 1), ('4457', 1)]
combo freq ==> [('[127175]', 241), ('[522484, 127175]', 55), ('[106546, 127175]', 53), ('[522484, 106546, 127175]', 49), ('[106546, 522484, 127175]', 18), ('[95987, 522484, 106546, 127175]', 13), ('[127175, 106546]', 12), ('[95987, 522484, 127175]', 10), ('[522484, 127175, 106546]', 7), ('[4483, 522484, 106546, 127175]', 4), ('[95987, 106546, 522484, 127175]', 4), ('[106546, 4483, 522484, 127175]', 3), ('[127175, 522484, 106546]', 3), ('[95987, 522484, 127175, 106546]', 3), ('[127175, 522484]', 